In [12]:
import numpy as np
import re
import itertools
import collections
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation

In [15]:
M = np.load('word2vec.npy')
word2index = dict()
with open('word_lst.txt') as file:
    for counter, line in enumerate(file):
        word = line.strip()
        word2index[word] = counter

def word2vec(word):
    idx = word2index.get(word)
    if idx is None:
        return np.zeros(200)
    return M[idx]

        
def get_vec(word):
    try:
        retval = M[word2index[word]]
    except KeyError:
        retval = np.zeros(200)

In [156]:
def import_dataset_with_pos(filename):
    
    start_tag = '<S> <S>+BSTag'
    end_tag = '</S> </S>+ESTag'
    
    sentence_lst = [] # Sentence is a list word which is list of candidate roots
    sentence_correct_lst = []
    with open(filename) as file:
        start_parsing = False
        for line in file:
            if start_parsing:
                if line.startswith(end_tag):
                    start_parsing = False
                    sentence_lst.append(sentence)
                    sentence_correct_lst.append(sentence_correct)
                else:
                    if "*UNKNOWN" in line:
                        continue
                    roots = []
                    candidate_lst = line.split()[1:]
                    for parse in candidate_lst:
                        morphemes = parse.split("^DB")
                        root = morphemes[0].split("+")[0]
                        pos_tag = morphemes[-1].split("+")[1]
                        roots.append((root.lower(), pos_tag))
                    if roots:
                        sentence.append(roots)
                        sentence_correct.append(roots[0])
                        
                
            else:
                if line.startswith(start_tag):
                    start_parsing = True
                    sentence = []
                    sentence_correct = []
                    
        return sentence_correct_lst, sentence_lst

In [157]:
train_dataset_filename = 'dataset/train.merge'
train_sentence_correct_lst, train_sentence_lst = import_dataset_with_pos(train_dataset_filename)

In [158]:
POS_SET = {'Dup', '', 'Interj', 'Punc', 'Nom', 'Pron', 'Adverb', 'Adj', 'Conj', 'Ques', 
           'Verb', 'Noun', 'A3sg', 'Num', 'Det', 'Postp', 'Adv'}
POS_2_ENCODED = {tag: idx for idx, tag in enumerate(POS_SET)}
max_idx = len(POS_2_ENCODED)
POS_2_ENCODED = {tag: np.eye(max_idx)[i] for tag, i in POS_2_ENCODED.items()}

In [159]:

model = Sequential([
    Dense(100, activation='relu', input_shape=(400+2*len(POS_2_ENCODED),)),
    Dense(100, activation='relu'),
    Dense(2),
    Activation('softmax'),
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

def generate_samples(sentences):
    for sentence in sentences:
        for w1, w2 in zip(sentence, sentence[1:]):
            itert = itertools.product(w1,w2)
            yield next(itert), 1
            for others in itert:
                yield others, 0
samples = list(generate_samples(train_sentence_lst))

In [166]:
def gen_subsamples():
    subsample_size = 10000
    for subsample in range(0, len(samples), subsample_size):
        subsamples = samples[subsample: subsample + subsample_size]
        train_data = np.array([
            np.append(np.append(word2vec(w1), POS_2_ENCODED[w1_pos]), np.append(word2vec(w2), POS_2_ENCODED[w2_pos]))
            for ((w1,w1_pos),(w2, w2_pos)),_ in subsamples])
        train_labels = np.array([[v==0, v==1] for _, v in subsamples])
        print(subsample, len(samples))
        yield train_data, train_labels
#train_data, train_labels = next(gen_subsamples())
#model.fit(train_data, train_labels, epochs=10, batch_size=32)
for train_data, train_labels in gen_subsamples():
    model.fit(train_data, train_labels, epochs=10, batch_size=64)

0 2738073
Epoch 1/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.5125 - acc: 0.7823
Epoch 2/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4700 - acc: 0.7913
Epoch 3/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4510 - acc: 0.8000
Epoch 4/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.4400 - acc: 0.8080
Epoch 5/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4299 - acc: 0.8114
Epoch 6/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4150 - acc: 0.8133
Epoch 7/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.4119 - acc: 0.8211
Epoch 8/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4018 - acc: 0.8215
Epoch 9/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.3938 - acc: 0.8270
Epoch 10/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.3892 - a

10000/10000 [==============================] - 0s 30us/step - loss: 0.4987 - acc: 0.7862
Epoch 2/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4595 - acc: 0.7983
Epoch 3/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4394 - acc: 0.8045
Epoch 4/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4270 - acc: 0.8106
Epoch 5/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.4164 - acc: 0.8183
Epoch 6/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.4056 - acc: 0.8231
Epoch 7/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4005 - acc: 0.8278
Epoch 8/10
10000/10000 [==============================] - 0s 30us/step - loss: 0.3942 - acc: 0.8276
Epoch 9/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.3874 - acc: 0.8331
Epoch 10/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.3839 - acc: 0.8314
90000 2738

10000/10000 [==============================] - 0s 32us/step - loss: 0.5260 - acc: 0.7873
Epoch 2/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4917 - acc: 0.7981
Epoch 3/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4680 - acc: 0.8049
Epoch 4/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4456 - acc: 0.8126
Epoch 5/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.4312 - acc: 0.8156
Epoch 6/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.4178 - acc: 0.8222
Epoch 7/10
10000/10000 [==============================] - 0s 34us/step - loss: 0.4069 - acc: 0.8248
Epoch 8/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.3990 - acc: 0.8278
Epoch 9/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.3948 - acc: 0.8321
Epoch 10/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.3851 - acc: 0.8319
170000 273

10000/10000 [==============================] - 0s 32us/step - loss: 0.4904 - acc: 0.7940
Epoch 2/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.4568 - acc: 0.8040
Epoch 3/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4419 - acc: 0.8107
Epoch 4/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.4316 - acc: 0.8171
Epoch 5/10
10000/10000 [==============================] - 0s 35us/step - loss: 0.4219 - acc: 0.8216
Epoch 6/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.4130 - acc: 0.8250
Epoch 7/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.4058 - acc: 0.8295
Epoch 8/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.3985 - acc: 0.8358
Epoch 9/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.3936 - acc: 0.8347
Epoch 10/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.3946 - acc: 0.8385
250000 273

10000/10000 [==============================] - 0s 32us/step - loss: 0.4953 - acc: 0.7869
Epoch 2/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.4599 - acc: 0.7997
Epoch 3/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4400 - acc: 0.8065
Epoch 4/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4304 - acc: 0.8145
Epoch 5/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4221 - acc: 0.8175
Epoch 6/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4135 - acc: 0.8202
Epoch 7/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4045 - acc: 0.8224
Epoch 8/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.3982 - acc: 0.8268
Epoch 9/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.3918 - acc: 0.8299
Epoch 10/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.3870 - acc: 0.8329
330000 273

...

10000/10000 [==============================] - 0s 32us/step - loss: 0.6172 - acc: 0.7578
Epoch 2/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.5685 - acc: 0.7722
Epoch 3/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.5477 - acc: 0.7781
Epoch 4/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.5326 - acc: 0.7790
Epoch 5/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.5176 - acc: 0.7858
Epoch 6/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.5140 - acc: 0.7921
Epoch 7/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.5048 - acc: 0.7929
Epoch 8/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4981 - acc: 0.7977
Epoch 9/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4942 - acc: 0.8009
Epoch 10/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4913 - acc: 0.8000
2410000 27

10000/10000 [==============================] - 0s 32us/step - loss: 0.5425 - acc: 0.7843
Epoch 2/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.5207 - acc: 0.7913
Epoch 3/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4959 - acc: 0.7963
Epoch 4/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4833 - acc: 0.8024
Epoch 5/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4806 - acc: 0.8054
Epoch 6/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4695 - acc: 0.8061
Epoch 7/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4613 - acc: 0.8114
Epoch 8/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4575 - acc: 0.8148
Epoch 9/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4539 - acc: 0.8195
Epoch 10/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4485 - acc: 0.8193
2490000 27

10000/10000 [==============================] - 0s 31us/step - loss: 0.4789 - acc: 0.7776
Epoch 2/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4626 - acc: 0.7889
Epoch 3/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4480 - acc: 0.7888
Epoch 4/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4350 - acc: 0.7969
Epoch 5/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4246 - acc: 0.8034
Epoch 6/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4182 - acc: 0.8116
Epoch 7/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4087 - acc: 0.8171
Epoch 8/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4030 - acc: 0.8188
Epoch 9/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.3984 - acc: 0.8212
Epoch 10/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.3922 - acc: 0.8253
2570000 27

10000/10000 [==============================] - 0s 32us/step - loss: 0.5436 - acc: 0.7660
Epoch 2/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.5147 - acc: 0.7707
Epoch 3/10
10000/10000 [==============================] - 0s 30us/step - loss: 0.5058 - acc: 0.7768
Epoch 4/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4913 - acc: 0.7826
Epoch 5/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4754 - acc: 0.7872
Epoch 6/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4720 - acc: 0.7918
Epoch 7/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.4661 - acc: 0.7947
Epoch 8/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4575 - acc: 0.8012
Epoch 9/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4569 - acc: 0.8044
Epoch 10/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.4478 - acc: 0.8073
2650000 27

10000/10000 [==============================] - 0s 31us/step - loss: 0.6395 - acc: 0.7593
Epoch 2/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.6039 - acc: 0.7669
Epoch 3/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.6033 - acc: 0.7710
Epoch 4/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.5850 - acc: 0.7772
Epoch 5/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.5827 - acc: 0.7794
Epoch 6/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.5758 - acc: 0.7837
Epoch 7/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.5615 - acc: 0.7931
Epoch 8/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.5563 - acc: 0.7929
Epoch 9/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.5487 - acc: 0.7989
Epoch 10/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.5459 - acc: 0.8004
2730000 27

In [167]:
model.save("pos_dense_100_100_2.keras")

In [31]:
model = keras.models.load_model("pos_dense_100_100_2.keras")

In [168]:
model.predict(np.array([np.append(np.append(
    word2vec("new"), POS_2_ENCODED["Noun"]), np.append(word2vec("york"), POS_2_ENCODED["Noun"]))]))

array([[0., 1.]], dtype=float32)

In [169]:
dataset_filename = 'dataset/test.merge'
sentence_correct_lst, sentence_lst = import_dataset(dataset_filename)

In [170]:
class PosScoreModel:
    def __init__(self, verbose=False):
        self.verbose = verbose
    
    def predict(self, sentence):
        predict_sentence = None
        self._cache = {}
        word_score = [[0]*10]
        backtrack = [[(0,0)]*10]
        for wi, (w_parses, w_next_parses) in enumerate(zip(sentence, sentence[1:])):
            word_score.append([0] * len(w_next_parses))
            backtrack.append([])
            for wn_i, wn in enumerate(w_next_parses):
                best_pair = None
                max_score = float('-inf')
                for wp_i, wp in enumerate(w_parses):
                    score = word_score[wi][wp_i] + self.calc_sentence_score((wp, wn))
                    word_score[wi+1][wn_i] = score
                    if score > max_score:
                        best_pair = (wp_i, wn_i)
                        max_score = score
                backtrack[-1].append(best_pair)
        if len(sentence) <2:
            return [sentence[0][0]]
        predict_sentence = [sentence[-1][best_pair[1]]]
        for w_parses, bt in zip(sentence[:-1][::-1], backtrack[:-1][::-1]):
            predict_sentence.append(w_parses[best_pair[0]])
            best_pair = bt[best_pair[0]]
        return predict_sentence[::-1]
    
    def pair_score(self, word1, word2):
        if (word1, word2) in self._cache:
            return self._cache[(word1, word2)]
        try:
            vec1 = M[word2index[word1[0]]]
            vec2 = M[word2index[word2[0]]]
        except:
            return 0
        #return np.abs(np.dot(vec1, vec2))
        prediction = model.predict(np.array([np.append(
            np.append(vec1, POS_2_ENCODED[word1[1]]), 
            np.append(vec2, POS_2_ENCODED[word2[1]]))]))[0]
        self._cache[(word1, word2)] = prediction[1]
        return self._cache[(word1, word2)]

    def calc_sentence_score(self, sentence):
        score = 0

        if len(sentence) <= 1:
            return score

        for i in range(len(sentence) - 1):
            word1 = sentence[i]
            word2 = sentence[i + 1]
            score += np.log(self.pair_score(word1, word2) + 1)

        return score

In [171]:
correct_count = 0
false_count = 0
correct_sentences = 0

mdl = PosScoreModel()
for num, (sentence, sentence_correct) in enumerate(zip(sentence_lst, sentence_correct_lst)):
    predict_sentence = mdl.predict(sentence)
    flag = True
    for word1, word2 in zip(predict_sentence, sentence_correct):
        if word1 == word2:
            correct_count += 1
        else:
            false_count += 1
            flag = False
    if flag:
        correct_sentences += 1

total_count = correct_count + false_count
accuracy = correct_count / total_count

print('Total number of words : %s' % total_count)
print('Correctly predicted : %s' % correct_count)
print('Correctly predicted sentences : %s' % correct_sentences)
print('Accuracy : %.3f' % accuracy)

Total number of words : 861
Correctly predicted : 801
Correctly predicted sentences : 16
Accuracy : 0.930


In [172]:
def parse_sentence(sentence):
    sentence = '\'' + sentence + '\''
    word_lst = !./trnltk/parser.py {sentence}
    #print(word_lst)
    retval = []
    for word in word_lst:
        root_lst = word.split()
        retval.append(root_lst)
    return retval

    yüklenen yükle+Verb^DB+Verb+Pass+Pos^DB+Adj+PresPart yük+Noun+A3sg+Pnon+Nom^DB+Verb+Acquire+Pos^DB+Adj+PresPart

    Sentence Examples
    
    dolar fiyatları beş TL seviyesinde bulunurken Euro fiyatları altı TL seviyesinde hareket ediyor
    yorulunca alın damarları gözükmeye başladı
    istediğiniz kadar ürün alın 
    sözlerine çok alındı (Wrong)
    ya iyi olarak ölürsün ya da kötüye dönüşecek kadar uzun yaşarsın
    nedir amacımız bunu göndermekle uzaylılara karsı bir sinerji yaratalım dostluk olsun mu
    kafanızı kullansaydınız o taşların doğada bulunan 4 elementi simgelediğini anlardınız

In [173]:
sentence = 'istediğiniz kadar ürün alın'
parsed_sentence = parse_sentence(sentence)

combination_count = 1
for word in parsed_sentence:
  combination_count *= len(word)
print('Total number of possibilities : %s' % combination_count)

print(parsed_sentence)
print(sentence)

vmodel = ScoreModel(verbose=True)
prediction = vmodel.predict(parsed_sentence)

print('-------')
print('Predicted roots : ')
print(prediction)

Total number of possibilities : 7
[['/bin/bash:', './trnltk/parser.py:', 'No', 'such', 'file', 'or', 'directory']]
istediğiniz kadar ürün alın
-------
Predicted roots : 
['/bin/bash:']
